## Group16 
## Xinmai Xuan, Peng Lu, Xiao Zhang

##  How to use the dashboard 
Our group uses two different dashboards to help those who are not experts understand our content, using data from ten countries as examples in our dataset. 

1. The **first dashboard** shows the relationship between Gross increase and Use of intenstiy in forests. By selecting single or multiple choices of the Gross increase in these countries for different years, you can find the forest's use of intensity in the bar chart on the right. This Dashboard allows you to see if the forests are over-exploited or if the normal trend is ensured. 

2. The **second Dashboard** reflects the relationship between Gross increase of forests and the factors of greenhouse gases which includes CO,CO2,PM2.5,etc. This dashboard provides information on the gross increase  of forests in each country and analyze whether the increase of forests have an impact on the change in emissions of these gases and PM2.5.For dashboard 2, year slider controls the europe map. The other two dropdown buttons control plot. Users can select their preferred country and type of greenhouse gas to check its variation. 

## Contextual datasets
In this project, we use 2 contextual datasets. 
1. The first one, named pollution exposure, reflects the PM 2.5 emission amount of each country. 
2. The other one, named GHG emissions, records values of some major greenhouse gas emissions such as CO2, CO, SOX, VOC, NOX, and GHG. 

We believe the variations of forest-related resources may influence the emission of greenhouse gases.  To explore their potential connections, we depict a dashboard to show the relation between countries' GROSSINCREASE value and those emissions.

## How large is the dataset
The maximum size of a single file on Github is 100MB, and all datasets we use in our final project do not exceed the capacity of Github. Therefore, We don't think a larger dataset or a plan to change the dataset will be applied to our fianl project.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import numpy as np
import contextily as ctx
import ipywidgets

In [2]:
# read dataset
# Air and GHG emissions
CO2 = pd.read_csv('https://raw.githubusercontent.com/lovefade/lovefade.github.io/main/data/CO2.csv')
CO = pd.read_csv('https://raw.githubusercontent.com/lovefade/lovefade.github.io/main/data/CO.csv')
GHG = pd.read_csv('https://raw.githubusercontent.com/lovefade/lovefade.github.io/main/data/GHG.csv')
NOx = pd.read_csv('https://raw.githubusercontent.com/lovefade/lovefade.github.io/main/data/NOx.csv')
SOx = pd.read_csv('https://raw.githubusercontent.com/lovefade/lovefade.github.io/main/data/SOx.csv')

# Air pollution exposure
PM25 = pd.read_csv('https://raw.githubusercontent.com/lovefade/lovefade.github.io/main/data/PM2.5.csv')

# forest resourses
Gross_increase = pd.read_csv('https://raw.githubusercontent.com/lovefade/lovefade.github.io/main/data/Gross_increase.csv')
Use_intensity = pd.read_csv('https://raw.githubusercontent.com/lovefade/lovefade.github.io/main/data/Intensity_of_use.csv')

In [3]:
merge1 = pd.merge(CO2, CO, on=["LOCATION", "TIME"],how='outer')
merge1 = merge1[['LOCATION','TIME','Value_x','Value_y']].rename(columns={"Value_x": "CO2", "Value_y": "CO"}, errors="raise")

In [4]:
merge2 = pd.merge(merge1, GHG, on=["LOCATION", "TIME"],how='outer').rename(columns={"Value": "GHG"})
merge3 = pd.merge(merge2, NOx, on=["LOCATION", "TIME"],how='outer').rename(columns={"Value": "NOx"})
merge4 = pd.merge(merge3, SOx, on=["LOCATION", "TIME"],how='outer').rename(columns={"Value": "SOx"})
merge4 = merge4[['LOCATION','TIME','CO2','CO','GHG','NOx','SOx']]
merge5 = pd.merge(merge4, PM25, on=["LOCATION", "TIME"],how='outer').rename(columns={"Value": "PM25"})
merge6 = pd.merge(merge5, Gross_increase, on=["LOCATION", "TIME"],how='outer').rename(columns={"Value": "Gross_increase"})
merge7 = pd.merge(merge6, Use_intensity, on=["LOCATION", "TIME"],how='outer').rename(columns={"Value": "Use_intensity"})
total_merged = merge7[['LOCATION','TIME','CO2','CO','GHG','NOx','SOx','PM25','Gross_increase','Use_intensity']]
# total_merged

In [5]:
# convert time to datetime type
total_merged['TIME'] = pd.to_datetime(total_merged['TIME'], format='%Y', errors='coerce').dt.year

C:\Users\Xinmai Xuan\AppData\Local\Temp\ipykernel_13104\1333799875.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_merged['TIME'] = pd.to_datetime(total_merged['TIME'], format='%Y', errors='coerce').dt.year


In [6]:
# filter a few countries in Europe and time from 2010 to 2020
total = total_merged[total_merged['LOCATION'].isin(['AUT', 'DNK', 'FRA', 'DEU','ESP','GBR','IRL','ITA','POL','PRT','NED'])]
total = total[(total['TIME']> 2010) & (total['TIME'] < 2020)].reset_index(drop=True)
# total
data = total

## Dashboard 2

In [7]:
eubd = geopandas.read_file('https://raw.githubusercontent.com/lovefade/lovefade.github.io/main/euro-border.geojson')
# eubd

In [8]:
selectedCountry = ['AUT', 'DEU', 'DNK', 'ESP', 'FRA', 'GBR', 'IRL', 'ITA', 'POL', 'PRT']
seleu = eubd[eubd["iso3"].isin(selectedCountry)]
# seleu.head()

In [9]:
forest = pd.read_csv('https://raw.githubusercontent.com/lovefade/lovefade.github.io/main/data/forest.csv')
selforest = forest[(forest['LOCATION'].isin(selectedCountry)) & (forest['SUBJECT'] == 'GROSSINCREASE')]
# selforest

In [10]:
data_map = (
    seleu.merge(selforest, left_on="iso3", right_on="LOCATION")[
        ["iso3", "geometry", "TIME", "Value"]
    ]
)

# data_map

## Interaction

In [11]:
@ipywidgets.interact(year_selector=ipywidgets.IntSlider(min=2000, max=2020, step=1, value=2000), country_selector = ipywidgets.Dropdown(options = selectedCountry))

def msp_create(year_selector, country_selector):
    curval = data_map[data_map["TIME"] == year_selector]
    fig,ax = plt.subplots(figsize=(20, 20))
    ax.set_xlim(-15, 27)
    ax.set_ylim(35, 62.25)

    curval.plot(ax=ax, alpha=0.5, edgecolor='k',column='Value',categorical=True, legend=True)

    ctx.add_basemap(ax=ax, crs=eubd.crs)
    plt.show()

interactive(children=(IntSlider(value=2000, description='year_selector', max=2020, min=2000), Dropdown(descrip…

In [12]:
selectedType = ['CO2', 'CO','GHG','NOx','SOx','PM25']
selectedCountry = ['AUT', 'DNK', 'FRA', 'DEU','ESP','GBR','IRL','ITA','POL','PRT']

In [13]:
@ipywidgets.interact(country = ipywidgets.Dropdown(options = selectedCountry),
                     type_selector = ipywidgets.Dropdown(options = selectedType))
def line_chart(country, type_selector):
    airtype = pd.pivot_table(data, values = type_selector, index=['LOCATION'], columns=['TIME'])
    plot_data = airtype[airtype.index == country]
    x, y = plot_data.columns, plot_data.iloc[0,:]
    plt.plot(x, y)
    plt.title(f'Line Chart of {type_selector} in {country}')
    plt.xlabel('time')
    plt.ylabel('value')
    plt.show()

interactive(children=(Dropdown(description='country', options=('AUT', 'DNK', 'FRA', 'DEU', 'ESP', 'GBR', 'IRL'…

In [14]:
@ipywidgets.interact(year_selector=ipywidgets.IntSlider(min=2000, max=2020, step=1, value=2000),
                     country = ipywidgets.Dropdown(options = selectedCountry),
                     type_selector = ipywidgets.Dropdown(options = selectedType))

def msp_line(year_selector):
    curval = data_map[data_map["TIME"] == year_selector]
    fig,ax = plt.subplots(figsize=(10, 10))
    ax.set_xlim(-15, 27)
    ax.set_ylim(35, 62.25)

    curval.plot(ax=ax, alpha=0.5, edgecolor='k',column='Value',categorical=True, legend=True)

    ctx.add_basemap(ax=ax, crs=eubd.crs)
    plt.show()
    
def line_chart(country, type_selector):
    airtype = pd.pivot_table(data, values = type_selector, index=['LOCATION'], columns=['TIME'])
    plot_data = airtype[airtype.index == country]
    x, y = plot_data.columns, plot_data.iloc[0,:]
    plt.plot(x, y)
    plt.title(f'Line Chart of {type_selector} in {country}')
    plt.xlabel('time')
    plt.ylabel('value')
    plt.show()

interactive(children=(IntSlider(value=2000, description='year_selector', max=2020, min=2000), Output()), _dom_…

In [15]:
@ipywidgets.interact(year_selector=ipywidgets.IntSlider(min=2000, max=2020, step=1, value=2000),
                     country = ipywidgets.Dropdown(options = selectedCountry),
                     type_selector = ipywidgets.Dropdown(options = selectedType))

def dashboard_created(year_selector, country, type_selector):
    
    
    curval = data_map[data_map["TIME"] == year_selector]
    airtype = pd.pivot_table(data, values = type_selector, index=['LOCATION'], columns=['TIME'])
    plot_data = airtype[airtype.index == country]
    
    fig,ax = plt.subplots(2, 1, figsize=(7.5, 10))
    ax[0].set_xlim(-15, 27)
    ax[0].set_ylim(35, 62.25)
    curval.plot(ax=ax[0], alpha=0.5, edgecolor='k',column='Value',categorical=True, legend=True)
    ctx.add_basemap(ax=ax[0], crs=eubd.crs)
    
    x, y = plot_data.columns, plot_data.iloc[0,:]
    ax[1].plot(x, y)
    ax[1].set_title(f'Line Chart of {type_selector} in {country}')
    ax[1].set_xlabel('time')
    ax[1].set_ylabel('value')
    
    plt.show()

interactive(children=(IntSlider(value=2000, description='year_selector', max=2020, min=2000), Dropdown(descrip…